In [1]:
import nltk                                      # Natural Language processing. Para el análisis lintuistico de documentos. 
from nltk.stem.lancaster import LancasterStemmer # Permite transformar palabras para que sea mas entendible (letras de mas o errores de tipeo, ect).
stemmer = LancasterStemmer ()
import numpy
import tensorflow as tf
import tflearn                                   # Para la implementación de redes neuronales multicapa
import json
import random
import pickle                                    # Permite guardar el modelo y no cargarlo cada vez que se ejecute el programa
import csv
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open("estandar.json", encoding='utf-8') as archivo:
    datos = json.load(archivo)
    
# print(datos) #muestra de datos y prueba de carga correcta

In [3]:
from tensorflow.python.framework import ops

# El 'try' permite almacenar los parametros del modelo para proximas ejecuciones. Se debe dejar comentado para las pruebas iniciales del sistema!!

########### Inicio comentado de almacenamiento ################################
# try: 
#     with open("variables.pickle","rb") as archivoPickle:
#         palabras, tags, entrenamiento, salida = pickle.load(archivoPickle)

# except:
###############################################################################

palabras=[]
tags=[]
auxX=[]  # palabras que se asignaran a la lista "palabras"
auxY=[]

for contenido in datos["cuentas"]:
    for patrones in contenido["patrones"]:
        auxPalabra = nltk.word_tokenize(patrones) # Toma una frase y la separa en palabras (tokens), para identificar los patrones a parametrizar
        palabras.extend(auxPalabra)               # Almacena todas las palabras tokenizadas 
        auxX.append(auxPalabra)                   
        auxY.append(contenido["tag"])

        if contenido["tag"] not in tags:          # Agrega un tag si no lo encuentra
            tags.append(contenido["tag"])

palabras = [stemmer.stem(w.lower()) for w in palabras if w != "\n"]   #stemmer permite identificar palabras equivalentes (por ejemplo: Banco = Bco. = banco)
palabras = sorted(list(set(palabras)))            # Se define la palabra
tags = sorted(tags)                               # Se define el tag

entrenamiento = []
salida = []
salidaVacia = [0 for _ in range(len(tags))]

for x, documento in enumerate(auxX):              # en X se guardara el indice de la palabra
    cubeta=[]                                     #Lista auxiliar para almacenar las palabras 
    auxPalabra = [stemmer.stem(w.lower()) for w in documento]
    for w in palabras:
        if w in auxPalabra:
            cubeta.append(1)
        else:
            cubeta.append(0)

    filaSalida = salidaVacia[:]
    filaSalida[tags.index(auxY[x])] = 1            #en la posicion del indice del tag, obtenemos el indice del auxiliar
    entrenamiento.append(cubeta)
    salida.append(filaSalida)

entrenamiento = numpy.array(entrenamiento)
salida = numpy.array(salida)

# ########### Inicio comentado de almacenamiento ##################################

#     with open("variables.pickle","wb") as archivoPickle:
#             pickle.dump((palabras, tags, entrenamiento, salida), archivoPickle)
            
# #################################################################################

In [4]:

ops.reset_default_graph()


    ### Creación red neuronal inicial de 4 redes, con 10 neuronas c/u

red = tflearn.input_data(shape=[None,len(entrenamiento[0])]) # Datos de entrada a entrenar
red = tflearn.fully_connected(red,10)                        # Se crean capas conectadas con 10 unidades/neuronas
red = tflearn.fully_connected(red,10)
#red = tflearn.fully_connected(red,10)
red = tflearn.fully_connected(red,len(salida[0]),activation="softmax") # función de activación de las capas ocultas
red = tflearn.regression(red, optimizer='adam', loss='categorical_crossentropy') #permite obtener la probabilidad de que tan eficaz es la clasificacion de los tag

modelo = tflearn.DNN(red)
#try:
#    modelo.load("modelo.tflearn")
#except:

historial = modelo.fit(entrenamiento,salida,n_epoch=400,batch_size=40,show_metric=True) #Entrenamiento para el proceso de modelado

modelo.save("modelo.tflearn")


Training Step: 18799  | total loss: 0.57299 | time: 0.268s
| Adam | epoch: 400 | loss: 0.57299 - acc: 0.9109 -- iter: 1840/1856
Training Step: 18800  | total loss: 0.53605 | time: 0.268s
| Adam | epoch: 400 | loss: 0.53605 - acc: 0.9098 -- iter: 1856/1856
--
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.data-00000-of-00001
INFO:tensorflow:200
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.index
INFO:tensorflow:200
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.meta
INFO:tensorflow:300


In [55]:
from fpdf import FPDF

def agregarPDF(cuentas, respuesta):

    pdf = FPDF(orientation = 'L', unit = 'mm', format='A4') 
    pdf.add_page()

    pdf.add_font('DejaVu', '', 'DejaVuSansCondensed.ttf', uni=True)
    pdf.set_font('DejaVu', '', 11)

    print(respuesta[0])
    # TEXTO
    #pdf.set_font('Arial', '', 10)

    # titulo
    pdf.cell(w = 0, h = 15, txt = 'Balance General', border = 1, ln=1, align = 'C', fill = 0)
    # encabezado


    pdf.cell(w = 30, h = 15, txt = 'Cuenta', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Activo', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Pasivo', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Perdida', border = 1, align = 'C', fill = 0)
    pdf.cell(w = 30, h = 15, txt = 'Ganancia', border = 1, align = 'C', fill = 0)
    
    pdf.multi_cell(w = 0, h = 15, txt = 'Descripcion', border = 1, align = 'C', fill = 0)


    #Tag: nombres de las cuentas resultantes de la estandarizacion
    #Datos
    #print(tag + "---------------- " + datos["nombre_cliente"])
    cont = 0
    for row in cuentas:
        #print(row)
        #print(respuesta[cont])
        if(cont < len(respuesta)):            
            char = respuesta[cont]
            
        cont = cont + 1
        perdida = row["perdida"]
        #pdf.cell(w = 30, h = 9, txt = char, border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 7, txt = row["id_estandar"], border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 7, txt = row["activo"], border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 7, txt = row["pasivo"], border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 7, txt = row["perdida"], border = 1, align = 'C', fill = 0)
        pdf.cell(w = 30, h = 7, txt = row["ganancia"], border = 1, align = 'C', fill = 0)

        pdf.multi_cell(w = 0, h = 7, txt = char, border = 1, align = 'C', fill = 0)
        
    
    #pdf.output('hoja.pdf')
    pdf.output("hoja.pdf", 'F')

In [60]:
with open('datos_entrenamiento/balance_puerto_lapiz.csv', newline='') as f:
    reader = csv.DictReader(f,delimiter=';')
    
    cuentas = []
    for row in reader: 
        cuentas.append(row) 
        #balance_salida.append(row)
        #print(row)


In [61]:

def mainBot():
    respuesta = []  # Temporal !!!
    
    #entrada = input("Usuario: ")
    count = 1   ############### Provisorio
    for x in range(len(cuentas) - 5):
        #print("-------------------",count,"-------------------")       ############### Provisorio
        aux = cuentas[x]
        indice = aux["nombre_cliente"]
        count = count + 1    ############### Provisorio
        cubeta = [0 for _ in range(len(palabras))]
        entradaProcesada = nltk.word_tokenize(indice)
        entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entradaProcesada]
        for palabraIndividual in entradaProcesada:
            for i,palabra in enumerate(palabras):
                if palabra == palabraIndividual:
                    cubeta[i] = 1
        resultados = modelo.predict([numpy.array(cubeta)])
        resultadosIndices = numpy.argmax(resultados)
        tag = tags[resultadosIndices]
        respuesta.append(tag)
        
    agregarPDF(cuentas, respuesta)    
    print("Estandarización completa!!")
    #agregarPDF(cuentas, respuesta)

mainBot()


Disponible
Estandarización completa!!
